# Stock Estimation based on portfolio size


## Introduction

#### Leveraging IEX Cloud, yfinance and a trading bot the project emphasies on creating a day trading bot. Data provided for this project by [IEX Cloud](https://iexcloud.io), [Yahoo Finance](https://pypi.org/project/yfinance/)


#### The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of stocks you should buy.

#### Credits and inspiration from:
* #### https://github.com/nickmccullum/algorithmic-trading-python


## Library Imports

#### We need to import the open-source software libraries

In [5]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from yahoo_fin.stock_info import get_data
import yahoo_fin.stock_info as si

## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [6]:
from secrets import IEX_CLOUD_API_TOKEN

## Parsing API Call


In [28]:
symbol = "aapl"
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()

## Importing List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis. [Yfinance](https://pypi.org/project/yfinance) was used to gather the data.

#### Let's import the stocks

In [7]:
dow_list = si.tickers_dow()
type(dow_list)

list

In [29]:
print("Tickers in Dow Jones:", len(dow_list))
dow_list[0:10]

Tickers in Dow Jones: 30


['AAPL', 'AMGN', 'AXP', 'BA', 'CAT', 'CRM', 'CSCO', 'CVX', 'DIS', 'DOW']

## Making Our First API Call

Structuring our api call to IEX

We'll need the following information from the API:

* Name of the stock
* Price
* Market capitalization for each stock
* Number of shares to buy
* IEX Close
* IEX Open
* Previous Close
* Previous Volume
* Actual Close
* Close Time



In [30]:
dow_historical = {}
for ticker in dow_list:
    dow_historical[ticker] = si.get_data(ticker, start_date="01/01/2020", end_date="01/03/2020", interval="1d")
dow_historical

{'AAPL':                  open       high        low      close   adjclose     volume  \
 2020-01-02  74.059998  75.150002  73.797501  75.087502  74.444603  135480400   
 
            ticker  
 2020-01-02   AAPL  ,
 'AMGN':              open        high         low       close    adjclose   volume  \
 2020-01-02  243.0  243.190002  238.979996  240.100006  233.642029  2088000   
 
            ticker  
 2020-01-02   AMGN  ,
 'AXP':                   open        high         low       close    adjclose  \
 2020-01-02  124.660004  126.269997  124.230003  125.849998  123.637741   
 
              volume ticker  
 2020-01-02  2708000    AXP  ,
 'BA':                   open        high         low       close    adjclose  \
 2020-01-02  328.549988  333.350006  327.700012  333.320007  331.348572   
 
              volume ticker  
 2020-01-02  4544400     BA  ,
 'CAT':              open        high         low       close    adjclose   volume  \
 2020-01-02  149.0  150.550003  147.979996  150.5

## Creating pandas columns

In [15]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy', "IEX Close", "IEX Open","Previous Close","Previous Volume","Close","Close Time"]
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time


## Looping Through Tickers


In [35]:
final_dataframe = pd.DataFrame(columns = my_columns)
for symbol in dow_list:
    api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    final_dataframe = final_dataframe.append(
                                        pd.Series([
                                                   symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)

In [36]:
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time
0,AAPL,134.860,2248947180561,N/A,132.260,135.910,138.01,149049774,None,None
1,AMGN,252.440,140957708609,N/A,244.120,252.330,258.67,5271388,None,None
2,AXP,116.920,96389317365,N/A,121.220,121.260,122.27,5867664,None,None
3,BA,198.430,111447506488,N/A,200.270,201.580,200.28,10749660,None,None
4,CAT,191.260,103706879879,N/A,188.680,191.110,186.00,3456262,None,None
5,CRM,236.100,215673805492,N/A,228.600,231.600,233.80,7474862,None,None
6,CSCO,45.660,193346043322,N/A,45.020,46.480,47.38,21956103,None,None
7,CVX,88.400,162642151554,N/A,85.410,86.700,93.41,8479984,None,None
8,DIS,172.460,316183807441,N/A,175.170,172.510,177.25,14929682,None,None
9,DOW,53.100,39609339525,N/A,51.990,54.390,56.39,5920004,None,None


In [37]:
final_dataframe.shape

(30, 10)

## Using Batch API Calls to Improve Performance

We will split our list of stocks into groups of 5

In [32]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [39]:
symbol_groups = list(chunks(final_dataframe['Ticker'], 5))
symbol_strings = []

for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
    print(symbol_strings[i])




AAPL,AMGN,AXP,BA,CAT
CRM,CSCO,CVX,DIS,DOW
GS,HD,HON,IBM,INTC
JNJ,JPM,KO,MCD,MMM
MRK,MSFT,NKE,PG,TRV
UNH,V,VZ,WBA,WMT


In [53]:

final_dataframe_2 = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data_b = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe_2 = final_dataframe.append(
                                        pd.Series([
                                                   symbol, 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A',
                                                    data['iexClose'],
                                                    data['iexOpen'],
                                                    data['previousClose'],
                                                    data['previousVolume'],
                                                    data['close'],
                                                    data['closeTime']
                                                        ],
                                                  index = my_columns), 
                                        ignore_index = True)
        

## Calculating the Number of Shares to Buy

In [54]:
portfolio_size = input("Enter the value of your portfolio:")

try:
    val = float(portfolio_size)
except ValueError:
    print("That's not a number! \n Try again:")
    portfolio_size = input("Enter the value of your portfolio:")

Enter the value of your portfolio:10000000


In [55]:
position_size = float(portfolio_size) / len(final_dataframe_2.index)
for i in range(0, len(final_dataframe_2['Ticker'])-1):
    final_dataframe_2.loc[i, 'Number Of Shares to Buy'] = math.floor(position_size / final_dataframe_2['Price'][i])
print(math.floor(position_size))
final_dataframe_2.head(10)

322580


,Ticker,Price,Market Capitalization,Number Of Shares to Buy,IEX Close,IEX Open,Previous Close,Previous Volume,Close,Close Time
0,AAPL,134.86,2248947180561,2391,132.26,135.91,138.01,149049774,None,None
1,AMGN,252.44,140957708609,1277,244.12,252.33,258.67,5271388,None,None
2,AXP,116.92,96389317365,2758,121.22,121.26,122.27,5867664,None,None
3,BA,198.43,111447506488,1625,200.27,201.58,200.28,10749660,None,None
4,CAT,191.26,103706879879,1686,188.68,191.11,186.00,3456262,None,None
5,CRM,236.10,215673805492,1366,228.60,231.60,233.80,7474862,None,None
6,CSCO,45.66,193346043322,7064,45.02,46.48,47.38,21956103,None,None
7,CVX,88.40,162642151554,3649,85.41,86.70,93.41,8479984,None,None
8,DIS,172.46,316183807441,1870,175.17,172.51,177.25,14929682,None,None
9,DOW,53.10,39609339525,6074,51.99,54.39,56.39,5920004,None,None


### Number of shares to buy for each stock

## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [58]:
writer = pd.ExcelWriter('recommended_trades.xlsx', engine='xlsxwriter')
final_dataframe_2.to_excel(writer, sheet_name='Recommended Trades', index = False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [57]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_format #This applies the format 'string_format' to the column
                    )
```

In [ ]:
# writer.sheets['Recommended Trades'].write('A1', 'Ticker', string_format)
# writer.sheets['Recommended Trades'].write('B1', 'Price', string_format)
# writer.sheets['Recommended Trades'].write('C1', 'Market Capitalization', string_format)
# writer.sheets['Recommended Trades'].write('D1', 'Number Of Shares to Buy', string_format)
# writer.sheets['Recommended Trades'].set_column('A:A', 20, string_format)
# writer.sheets['Recommended Trades'].set_column('B:B', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('C:C', 20, dollar_format)
# writer.sheets['Recommended Trades'].set_column('D:D', 20, integer_format)


This code works, but it violates the software principle of "Don't Repeat Yourself". 

Let's simplify this by putting it in 2 loops:

In [60]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format],
                    'E': ['IEX Close' , integer_format],
                    'F': ['IEX Open', integer_format],
                    'G': ['Previous Close', integer_format],
                    'H': ['Previous Volume', integer_format],
                    'I': ['Actual Close', integer_format],
                    'J': ['Close Time', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommended Trades'].write(f'{column}1', column_formats[column][0], string_format)

## Saving Our Excel Output

Saving our Excel file is very easy:

In [61]:
writer.save()